# K-fold Cross Validation
## Question - 1: 

K-fold CV is used to evaluate ML model performance. Here, we split the data into k subsets or folds of equal size. We use k-1 of the subsets as the training data and use the last fold as the test or validation set. We repeat this process k times so that each fold becomes the test set once. For each of these runs we collect a test error metric like RMSE, and at the end we compute the average of these errors to estimate the performance of the model.

The validation set approach is essentially K-fold CV but we only do the first run. As a result, the model never gets to see the data in the test fold whereas in K-fold CV the model gets to see all the data in the training process at some point. As a result, it is not as robust of an estimate of model performance returning only one round of test error instead of an average over k rounds.

LOOCV is K-fold CV when k = n(the number of rows in the dataset). LOOCV is arguably the most robust estimate of model performance due to it averaging n test errors, but this method can be prohibitively expensive computationally. With a large dataset of n > 10,000+ the benefits of this method are substantially outweighed by the costs.

## Question - 2: 
### 2.1 - Generate and plot the data

In [21]:
set.seed(100)
X <- rnorm(500)
Y <- 5.3 + 12*X - 3.41*(X**2)
xy <- data.frame(X, Y)

### 2.2 - LOOCV using least squares

In [12]:
library(tidyverse)
library(modeldata)
library(leaps)
library(caret)
library(corrplot)

In [24]:
## Make CV function with specified models
CV <- function(data, k){
  cutoff <- floor(nrow(data)/k)
  mse_1 <- c()
  mse_2 <- c()
  mse_3 <- c()
  mse_4 <- c()
  for (i in 1:k){
    start <- 1 + ((i-1) * cutoff)
    stop <- start + cutoff - 1
    train <- data[!1:nrow(data) %in% start:stop,]
    test <- data[start:stop,]
    model_1 <- lm(Y ~ X, data = train)
    model_2 <- lm(Y ~ poly(X,2,raw = T), data = train)
    model_3 <- lm(Y ~ poly(X,3,raw = T), data = train)
    model_4 <- lm(Y ~ poly(X,4,raw = T), data = train)
    predict_1 <- predict(model_1, test)
    predict_2 <- predict(model_2, test)
    predict_3 <- predict(model_3, test)
    predict_4 <- predict(model_4, test)
    mse_1 <- c(mse_1, RMSE(predict_1, test$Y))
    mse_2<- c(mse_2, RMSE(predict_2, test$Y))
    mse_3 <- c(mse_3, RMSE(predict_3, test$Y))
    mse_4 <- c(mse_4, RMSE(predict_4, test$Y))
  }
  return(c("Model 1" = round(mean(mse_1),4), "Model 2" = round(mean(mse_2),4), "Model 3" = round(mean(mse_3),4),
  "Model 4" = round(mean(mse_4),4)))
}
CV(xy, k = nrow(xy))


Model 1 Model 2 Model 3 Model 4 
 3.5299  0.0000  0.0000  0.0000

### 2.3 - 5-fold CV using least squares

In [37]:
CV(xy, k = 5)

Model 1 Model 2 Model 3 Model 4 
 5.3082  0.0000  0.0000  0.0000

### 2.4 - Which model had lowest LOOCV error? As expected? Comment on statistical significance of coefficients and do these results agree with the CV results?

In [34]:
## Fitting on first 400 as the "train" data - random sampling doesn't matter because they are all generated randomly
model_1 <- lm(Y ~ X, data = xy[1:400,])
model_2 <- lm(Y ~ poly(X,2,raw = T), data = xy[1:400,])
model_3 <- lm(Y ~ poly(X,3,raw = T), data = xy[1:400,])
model_4 <- lm(Y ~ poly(X,4,raw = T), data = xy[1:400,])

In [43]:
print("Model 1 coefficients:")
print(summary(model_1))
print("Model 2 coefficients:")
print(summary(model_2))
print("Model 3 coefficients:")
print(summary(model_3))
print("Model 4 coefficients:")
print(summary(model_4))

[1] "Model 1 coefficients:"

Call:
lm(formula = Y ~ X, data = xy[1:400, ])

Residuals:
    Min      1Q  Median      3Q     Max 
-34.052  -0.817   2.074   3.000   3.317 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)   1.9826     0.2602    7.62 1.87e-13 ***
X            12.0425     0.2637   45.66  < 2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 5.203 on 398 degrees of freedom
Multiple R-squared:  0.8397,	Adjusted R-squared:  0.8393 
F-statistic:  2085 on 1 and 398 DF,  p-value: < 2.2e-16

[1] "Model 2 coefficients:"


Warning message in summary.lm(model_2):
"essentially perfect fit: summary may be unreliable"



Call:
lm(formula = Y ~ poly(X, 2, raw = T), data = xy[1:400, ])

Residuals:
       Min         1Q     Median         3Q        Max 
-1.105e-14 -8.400e-16 -1.570e-16  5.730e-16  4.818e-14 

Coefficients:
                       Estimate Std. Error    t value Pr(>|t|)    
(Intercept)           5.300e+00  1.853e-16  2.860e+16   <2e-16 ***
poly(X, 2, raw = T)1  1.200e+01  1.583e-16  7.582e+16   <2e-16 ***
poly(X, 2, raw = T)2 -3.410e+00  1.026e-16 -3.325e+16   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 3.122e-15 on 397 degrees of freedom
Multiple R-squared:      1,	Adjusted R-squared:      1 
F-statistic: 3.447e+33 on 2 and 397 DF,  p-value: < 2.2e-16

[1] "Model 3 coefficients:"


Warning message in summary.lm(model_3):
"essentially perfect fit: summary may be unreliable"



Call:
lm(formula = Y ~ poly(X, 3, raw = T), data = xy[1:400, ])

Residuals:
       Min         1Q     Median         3Q        Max 
-1.101e-14 -8.130e-16 -1.500e-16  5.770e-16  4.817e-14 

Coefficients:
                       Estimate Std. Error    t value Pr(>|t|)    
(Intercept)           5.300e+00  1.856e-16  2.856e+16   <2e-16 ***
poly(X, 3, raw = T)1  1.200e+01  2.556e-16  4.695e+16   <2e-16 ***
poly(X, 3, raw = T)2 -3.410e+00  1.030e-16 -3.310e+16   <2e-16 ***
poly(X, 3, raw = T)3 -2.411e-17  5.976e-17 -4.040e-01    0.687    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 3.125e-15 on 396 degrees of freedom
Multiple R-squared:      1,	Adjusted R-squared:      1 
F-statistic: 2.293e+33 on 3 and 396 DF,  p-value: < 2.2e-16

[1] "Model 4 coefficients:"


Warning message in summary.lm(model_4):
"essentially perfect fit: summary may be unreliable"



Call:
lm(formula = Y ~ poly(X, 4, raw = T), data = xy[1:400, ])

Residuals:
       Min         1Q     Median         3Q        Max 
-1.107e-14 -8.160e-16 -2.310e-16  5.550e-16  4.798e-14 

Coefficients:
                       Estimate Std. Error    t value Pr(>|t|)    
(Intercept)           5.300e+00  2.111e-16  2.511e+16   <2e-16 ***
poly(X, 4, raw = T)1  1.200e+01  2.562e-16  4.683e+16   <2e-16 ***
poly(X, 4, raw = T)2 -3.410e+00  2.463e-16 -1.384e+16   <2e-16 ***
poly(X, 4, raw = T)3 -4.733e-17  6.083e-17 -7.780e-01   0.4370    
poly(X, 4, raw = T)4  6.721e-17  3.566e-17  1.885e+00   0.0602 .  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 3.115e-15 on 395 degrees of freedom
Multiple R-squared:      1,	Adjusted R-squared:      1 
F-statistic: 1.731e+33 on 4 and 395 DF,  p-value: < 2.2e-16



Models 2,3, and 4 all had 0 RMSE error, which at first shocked me, but now it makes sense looking at the variable significances. The data was based on a clear cut ground truth equation with 0 noise added to it. As a result, the models with the appropriate degree polynomials (2+) were able to map perfectly onto the data. Those with excess degrees simply put the coefficients at essentially 0 and were not significant, erasing their effects. This is what we see above looking at the model summaries, with the parameters mapping exactly to the ground truth and being significant up to $X^2$. Based on these model summaries, I can now see why the CV evaluations for models 2+ showed 0 error - because they mapped exactly correctly to the ground truth.

# Subset selection
## Question - 3:

Best subset selection means trying all possible subsets of features and selecting the best model based on metrics like AIC, BIC, and AdjR2, which are estimates of the test error. Stepwise subset selection follows this same approach of trying models and evaluating based on the same metrics, however the big difference is how it selects subsets. Stepwise subset selection does not try all possible models, it either adds or removes features one at a time based on what improves the model most. This means that instead of trying all possible models after a feature is added or removed, it will only try the models with one more or one less feature and see what does best, then choosing that model to work on next. Once it can no longer improve significantly it will stop. The two main types are forward and backward selection.

## Question - 4:

These metrics are all used to measure model fit. 

RSS (residual sum of squares) measures the squared difference between predicted and actual values. A lower RSS means better fit. This method emphasizes outliers due to the squaring, and thus will have a higher RSS than some other methods if there are a few predictions that are way off.

AIC balances model complexity with goodness of fit. It penalizes more parameters in a model and also takes into account the accuracy of the model. A model with low complexity and low RSS will have a low AIC.

BIC is similar to AIC but puts a harsher penalty on model complexity. It can also be more effective in smaller sample sizes as it is derived more rigorously from bayesian theory. Oftentimes it will have a minimum value for a less complex model than what other metrics would suggest.

$C_p$ also balances model complexity and goodness of fit, taking into account MSE and the number of parameters. It compares the MSE of the model to the SSE of a model with all the predictors.

## Question - 5:

RSS is a measure of total model fit irrespective of complexity. It is a blunt measure that only cares about the relationship between predictions and actual values. AIC, BIC, and $C_p$ all balance model complexity and performance. BIC is the strictest of these on model complexity and better in small sample sizes. AIC is the same as BIC but less harsh and less good under small sample sizes. $C_p$ uses the MSE to estimate accuracy which is different then the other methods, and balances MSE, parameters, and sample size to generate a score. It is important to note that AIC, BIC and $C_p$ are used to evaluate model performance while RSS is usually not.

## Question - 6:

$R^2$ represents the proportion of RSS/TSS or the proportion of the variance in target values that is explained by the features. The more variation the features explain the higher $R^2$ is because the formula is 1 - RSS/TSS. The values only span from 0-1. 

$R^2_{adj}$ is the same as $R^2$ except it adds extra parameters to adjust for the number of parameters, pushing the score lower as more are added. As a result, it is a better metric and yields models less prone to overfitting. However, they are equivalently useful metrics when evaluating models with the same number of parameters.

## Question - 7:

RSS and $R^2$ metrics are not suitable for subset selection procedures because these procedures rely on looking at models with different numbers of parameters and comparing them. These metrics do not take into account model complexity and thus provide little usefulness in pursuit of finding the best, simplest model. These metrics will simply show you the best model, but not the simplest. We want a simpler model because they are less prone to overfitting on unseen data.

The other grouping of metrics are suitable because they account for model complexity as well as accuracy. They each do so in their own way with BIC being the most strict on complexity, but they all impose a penalty for more parameters, the difference is only in how they impose it. As a result, when comparing models of different complexity, we use these metrics to examine the models. However, it is important to note that it is best practice to use more than one of these metrics when comparing models as they can, and often do, provide different results.

